# Aqueous speciation in the system Cu-Co-S-Cl-Na
In this notebook, I determine the speciation of Cu and Co-bearing complexes in a saline, S-bearing aqueous solution under hydrothermal conditions. 

## 1. Import Libraries

In [1]:
import numpy as np
import math 
import thermofun as fun
import reaktoro as rkt

## 2. Initialize a thermodynamic database; check the contents of the database

In [2]:
#database = fun.Database('./databases/ThermoFun_db/mines16-thermofun.json')
#database = fun.Database('./databases/Co_Cu_aq_speciation.json')
database = fun.Database('./databases/Willy_ThermoHub_db.json')

elements = database.mapElements()
print(elements.keys())
reactions = database.mapReactions()
print(reactions.keys())
substances = database.mapSubstances()
print(substances.keys())
engine = fun.ThermoEngine(database)


dict_keys(['Ag', 'Al', 'Ar', 'As', 'Au', 'B', 'Ba', 'Be', 'Bi', 'Br', 'C', 'Ca', 'Cd', 'Ce', 'Cl', 'Co', 'Cr', 'Cs', 'Cu', 'Dy', 'Er', 'Eu', 'F', 'Fe', 'Ga', 'Gd', 'H', 'He', 'Hf', 'Hg', 'Ho', 'I', 'In', 'K', 'Kr', 'La', 'Li', 'Lu', 'Mg', 'Mn', 'Mo', 'N', 'Na', 'Nb', 'Nd', 'Ne', 'Ni', 'Nia', 'O', 'P', 'Pb', 'Pd', 'Pr', 'Pt', 'Ra', 'Rb', 'Re', 'Rh', 'Rn', 'Ru', 'S', 'Sb', 'Sc', 'Se', 'Si', 'Sm', 'Sn', 'Sr', 'Tb', 'Tc', 'Th', 'Ti', 'Tl', 'Tm', 'U', 'V', 'W', 'Xe', 'Y', 'Yb', 'Zn', 'Zr', 'Zz'])
dict_keys(['Ce(SO4)2-', 'CeSO4+', 'Dy(SO4)2-', 'DySO4+', 'Er(SO4)2-', 'ErSO4+', 'Eu(SO4)2-', 'EuSO4+', 'FeSO4', 'Gd(SO4)2-', 'GdSO4+', 'H2WO4', 'Ho(SO4)2-', 'HoSO4+', 'KHWO4', 'KWO4-', 'La(SO4)2-', 'LaSO4+', 'Lu(SO4)2-', 'LuSO4+', 'MnCl2', 'MnHCO3+', 'NaCO3-', 'NaHCO3', 'NaHWO4', 'NaWO4-', 'Nd(OH)2+', 'Nd(OH)3', 'Nd(SO4)2-', 'NdOH++', 'NdSO4+', 'Pr(SO4)2-', 'PrSO4+', 'Sm(SO4)2-', 'SmSO4+', 'Sn(HCO3)2', 'Sn(HS)2', 'Sn(OH)2', 'Sn(OH)2++', 'Sn(OH)3+', 'Sn(OH)3-', 'Sn(OH)4', 'Sn(OH)4--', 'Sn(OH)5-', 'S

## 3. Set temperature and pressure conditions

In [3]:
T = 200 + 273.15 # [K]
P = rkt.waterSaturatedPressureWagnerPruss(T).val/100000 # [bar] saturated water vapor pressure

## 4. Set pH and chlorinity conditions

In [4]:
# Target pH 
pH = 3
# Target chlorinity
c_Cl = 1 # [mol*kg^-1]

## 5. Define and construct the chemical system

In [5]:
# Define the chemical system
editor = rkt.ChemicalEditor(database)
editor.addAqueousPhaseWithElements([
'H', 'O', 'Cl', 'Na', 'S', 'Co'])
# Construct the chemical system
system = rkt.ChemicalSystem(editor)

RuntimeError: 
********************************************************************
*** Error: Fromula Parser
*** Reason:  A symbol of element expected here!
*** Location: ...80702/work/ThermoFun/Common/formulaparser.cpp:294
********************************************************************



## 6. Define the chemical equilibrium problem

In [ ]:
problem = rkt.EquilibriumProblem(system)
problem.add("Co",0.03,"mol") 
problem.add("H2O",1,"kg")
problem.add("HCl",10**-pH,"mol")
problem.add("NaCl",c_Cl - 10**-pH,"mol")
problem.setPressure(P, "bar")
problem.setTemperature(T, "kelvin")

## 7. Calculate the chemical equilibrium state

In [ ]:
state = rkt.equilibrate(problem)
print(state)

In [ ]:
# T = 298.15 # [K]
# P = 1e5 # [Pa]
# dGr = rkt.engine.thermoPropertiesReaction(T, P, "CoCl4-2").reaction_gibbs_energy # [J*mol^-1]
# print(dGr.val) 